In [1]:
#全局变量——最小支持度
MIN_SUPPORT = 0.2
MIN_CONFIDENCE = 0.6
# DEFAULT_DATASET = ['abc', 'ab', 'bc', 'abcde', 'bcd', 'bde', 'abe', 'abce', 'abcd']

In [2]:
#组合新项集
def get_frequent_obj(obj1, obj2):
    l = list(obj1)
    obj = ''.join(l)
    sign = 0
    for char in obj2:
        if char not in obj1:
            obj += char
            break
    l = list(obj)
    l.sort()
    obj = ''.join(l)
#    print('组合新项集' + obj)
    return obj

In [3]:
#检查是否可作为k项集
def check(obj1, obj2):
    l = list(obj1)
    l.sort()
    obj1 = ''.join(l)
    
    l = list(obj2)
    l.sort()
    obj2 = ''.join(l)
    
#    print('检查' + obj1 + '和' + obj2 + '是否可加')
    
    #检测位数增一
    diff = 1
    temp1 = 0
    temp2 = 0
    while (diff >=0 and temp1 < len(obj1) and temp2 < len(obj2)):
        if obj1[temp1] == obj2[temp2]:
            temp1 += 1
            temp2 += 1
        elif obj1[temp1] < obj2[temp2]:
            temp1 += diff
            diff -= 1
            if diff < 0:
#                print('不可加')
                return False
        elif obj1[temp1] > obj2[temp2]:
            temp2 += diff
            diff -= 1
            if diff < 0:
#                print('不可加')
                return False
#    print('可加')
    return True

In [4]:
#检验支持度是否足够
def if_support(obj):
    sum = 0
    for string in DEFAULT_DATASET:
        for i in range(len(obj)):
            if obj[i] in string:
                if i==len(obj)-1:
                    sum += 1
                else:
                    pass
            else:
                break
    print(obj + '的支持度为：' + str(sum) + '/' + str(len(DEFAULT_DATASET)) + '=' + str(sum/len(DEFAULT_DATASET)))
    if sum/len(DEFAULT_DATASET) < MIN_SUPPORT:
        return False
    else:
        return True

In [5]:
#检查子集是否非频繁
def has_infrequent_subset(obj,length):
#    print('当前子集为：' + obj)
#    print('当前length:' + str(length))
    if length == -1:
        return False
    frequent_dict = frequent_list[length]
#    print(frequent_dict)
    for i in range(len(obj)):
        string = (obj * 2) [i : len(obj)*2] [:len(obj)-1]
#        print('获取子串：' + string)
        l = list(string)
        l.sort()
        string = "".join(l)
#        print('检查' + string + '的子集')
        if frequent_list[length][string] == 0:
#            print('子串中' + string + '为非频繁子串')
            return True
#        print('子串' + string + '传入length：' + str(length-1))
        if has_infrequent_subset(string, length-1) == True:
#            print('查询子串' + string + '的子串,传入length参数为' + str(length-1))
            return True
#    print('当前串频繁：' + obj)
    return False

In [6]:
#递推Lk——i为当前项集长度-1
def find_next_frequent_list(frequent_list,i):
    next_frequent_dict = {}
#    print('当前已有项集表:',frequent_list)
    for frequent_obj1 in frequent_list[i].keys():
        #该项集不频繁
        is_frequent1 = frequent_list[i].get(frequent_obj1, 0)
#        print('左：获取长度为'+ str(i+1) + '的项集：' + frequent_obj1)
#        print('左：该项集是否频繁：' + '否' if is_frequent1 == 0 else '是')
        if is_frequent1 == 0:
            continue
        for frequent_obj2 in frequent_list[i].keys():
            is_frequent2 = frequent_list[i].get(frequent_obj2, 0)
#            print('右：获取长度为'+ str(i+1) + '的项集：' + frequent_obj2)
#            print('右：该项集是否频繁：' + '否' if is_frequent2 == 0 else '是')
            
            if frequent_obj1 > frequent_obj2:
#                print('左大于右，跳过循环')
                continue
            
            if is_frequent2 == 0:
                continue
                
            if frequent_obj1 == frequent_obj2:
#                print('左等于右，跳过当前循环')
                continue
            else:
                pass
            
            #检查位数满足条件
            if check(frequent_obj1, frequent_obj2):
#                print(frequent_obj1 + '加上' + frequent_obj2 + '满足位数')
                #获取该项集
                new_frequent_obj = get_frequent_obj(frequent_obj1, frequent_obj2)
#                print('获得新项集：' + new_frequent_obj)

                if new_frequent_obj in next_frequent_dict:
                    continue
                
                #默认频繁
                default_frequent = 1
                #剪枝
                if if_support(new_frequent_obj) == False:
#                    print('自身非频繁')
                    default_frequent = 0
                else:
#                    print('检查子集')
                    if has_infrequent_subset(new_frequent_obj,i) == True:
#                        print('子集有非频繁项集')
                        default_frequent = 0
                    
                #加入Lk字典
                next_frequent_dict[new_frequent_obj] = default_frequent
#                print('加入新项集：' + new_frequent_obj)
            else:
#                print(frequent_obj1 + '加上' + frequent_obj2 + '不满足位数')
                pass
            
    print()
    #寻至后无频繁项集结束
    flag = 0
    for frequent_obj in next_frequent_dict.keys():
#        print('当前检查：' + frequent_obj)
        is_frequent = next_frequent_dict.get(frequent_obj)
        if is_frequent == 1:
#            print('有频繁项集，可继续寻找')
            flag = 1
            break
        else:
            pass
#    print('list增加新字典')
    frequent_list.append(next_frequent_dict)
    if flag == 0:
#        print('无频繁项集可增加')
        return frequent_list
    else:
        return find_next_frequent_list(frequent_list, i+1)

In [7]:
#检测关联规则
def get_confidence(taskA, taskB):
    
    l = list(taskA)
    l.sort()
    taskA = ''.join(l)
#     print('taskA=' + taskA)
    
    l = list(taskA + taskB)
    l.sort()
    taskAB = ''.join(l)
#     print('taskAB=' + taskAB)
    
    
    countA = 0
    countAB = 0
    
    for string in DEFAULT_DATASET:
        if_count = True
#         print('get_from_dataset:' + string)
        for char in taskA:
            if char not in string:
#                 print(char + '不在' + string + '中')
                if_count = False
                break
            else:
                pass
        if if_count == False:
            continue
        countA += 1
        
        if_count = True
        for char in taskB:
            if char not in string:
                if_count = False
                break
            else:
                pass
        if if_count == True:
            countAB += 1
        else:
            pass
    
#     print(str(taskA) + '和' + str(taskB) + '同时出现：' + str(countAB) + '次')
#     print(str(taskA) + '出现：' + str(countA) + '次')
    return countAB / countA


In [16]:
if __name__ == '__main__':
    
    DEFAULT_DATASET = []
    
    datafile = open('D:/jupyterWorkSpace/mushroom.dat')
    for line in datafile.readlines():
        datalist = line.split(' ')
#         print(datalist)
        for char in datalist:
            if '\n' in char:
                char = char[:len(char)-1]
#             print(char)
            if char not in DEFAULT_DATASET:
                DEFAULT_DATASET.append(char)
#     print(DEFAULT_DATASET)
    
    frequent_list = []
    frequent_dict = {}
    for string in DEFAULT_DATASET:
        if string not in frequent_dict.keys():
            if if_support(string):
                frequent_dict[string] = 1
            else:
                frequent_dict[string] = 0
#         for char in string:
#             if char not in frequent_dict.keys():
#                 if if_support(char):
#                     frequent_dict[char] = 1
#                 else:
#                     frequent_dict[char] = 0
#             else:
#                 pass

    
        
    frequent_list.append(frequent_dict)
    print('获得初始列表：', frequent_list)
    print()
    frequent_list = find_next_frequent_list(frequent_list,0)
    print('获得最终列表：', frequent_list)
#     for obj in frequent_list:
#         for key in obj.keys():
#             print(int(key))

    for key in frequent_list[-2].keys():
        if frequent_list[-2][key] == 1:
            print('获取' + key + '的关联规则：')
            for i in range(len(key)):
                taskA = (key * 2) [i : len(key) * 2] [:len(key)-1]
                taskB = (key * 2) [i : len(key) * 2] [len(key)-1]
                l = list(taskA)
                l.sort()
                taskA = ''.join(l)
#                 print(taskA)
#                 print(taskB)
                print(taskA + (' ' * (len(taskB) - len(taskA))) + ' -> ' + (' ' * (len(taskA) - len(taskB))) + taskB + ' = confidence(' + str(get_confidence(taskA, taskB)) + ')' )
                print(taskB + (' ' * (len(taskA) - len(taskB))) + ' -> ' + (' ' * (len(taskB) - len(taskA))) + taskA + ' = confidence(' + str(get_confidence(taskB, taskA)) + ')' )
                print()

1的支持度为：39/119=0.3277310924369748
3的支持度为：21/119=0.17647058823529413
9的支持度为：21/119=0.17647058823529413
13的支持度为：4/119=0.03361344537815126
23的支持度为：2/119=0.01680672268907563
25的支持度为：2/119=0.01680672268907563
34的支持度为：2/119=0.01680672268907563
36的支持度为：2/119=0.01680672268907563
38的支持度为：2/119=0.01680672268907563
40的支持度为：2/119=0.01680672268907563
52的支持度为：2/119=0.01680672268907563
54的支持度为：2/119=0.01680672268907563
59的支持度为：2/119=0.01680672268907563
63的支持度为：2/119=0.01680672268907563
67的支持度为：2/119=0.01680672268907563
76的支持度为：2/119=0.01680672268907563
85的支持度为：2/119=0.01680672268907563
86的支持度为：2/119=0.01680672268907563
90的支持度为：2/119=0.01680672268907563
93的支持度为：2/119=0.01680672268907563
98的支持度为：2/119=0.01680672268907563
107的支持度为：1/119=0.008403361344537815
113的支持度为：4/119=0.03361344537815126
2的支持度为：21/119=0.17647058823529413
14的支持度为：4/119=0.03361344537815126
26的支持度为：2/119=0.01680672268907563
39的支持度为：2/119=0.01680672268907563
55的支持度为：21/119=0.17647058823529413
99的支持度为：21/119=0.17647058823529413
108的支持度为：1

KeyError: ''